<a href="https://colab.research.google.com/github/siting1206/AIcup_NLP/blob/main/AICUP_train_datareduce_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## import package

In [1]:
!pip install torch
!pip install pandas nltk
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.6 MB/s 
     |████████████████████████████████| 7.6 MB 48.8 MB/s 
     |████████████████████████████████| 182 kB 68.8 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import math
import numpy as np
import time
import torch, pandas as pd
import nltk
import re
nltk.download('punkt')

from transformers import set_seed
set_seed(123)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cuda:0


In [4]:
# Training data file
directory="/content/drive/MyDrive/AICUP_NLP/"


file=directory+"train_reduce.csv"
df=pd.read_csv(file, encoding = "ISO-8859-1")

In [5]:
df.head()

,id,q,r,s,q',r'
0,8,"""It can go both ways . We all doubt . It is wh...","""True .""",AGREE,"""It can go both ways . We all doubt . It is wh...","""True"""
1,9,"""once again , you seem to support the killing ...","""based on the idea that people are dispensible...",AGREE,"""seem to support the killing of certain people""","""based on the idea that people are dispensible..."
2,10,"""I personly would not condone an abortion , ho...","""This is a pretty touchy issue , and I agree w...",AGREE,"""personly would not condone an abortion , howe...","""This is a pretty touchy issue , and I agree w..."
3,11,"""First , there is no `` us `` on your part reg...","""Ah , I see . Your reasons are secret reasons ...",DISAGREE,"""You should be giving justification for wantin...","""Your reasons are secret reasons . On a debate..."
4,12,"""alright how did evolution start ?""","""See above""",DISAGREE,"""alright how did evolution start ?""","""See above"""


## Data process

In [6]:
df[['q','r',"q'","r'"]] = df[['q','r',"q'","r'"]].apply(lambda x: x.str.strip('\"'))

In [7]:
df['r'] = df['s'] + ':' + df['r']

In [8]:
df['sub_q_true'] = [1 if x in y else 0 for x,y in zip(df["q'"],df["q"])]
df['sub_r_true'] = [1 if x in y else 0 for x,y in zip(df["r'"],df["r"])]
df['sub_both'] = df['sub_q_true']*df['sub_r_true']

In [9]:
df.head()

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1
1,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1
2,10,"I personly would not condone an abortion , how...","AGREE:This is a pretty touchy issue , and I ag...",AGREE,"personly would not condone an abortion , howev...","This is a pretty touchy issue , and I agree wi...",1,1,1
3,11,"First , there is no `` us `` on your part rega...","DISAGREE:Ah , I see . Your reasons are secret ...",DISAGREE,You should be giving justification for wanting...,Your reasons are secret reasons . On a debate ...,0,1,0
4,12,alright how did evolution start ?,DISAGREE:See above,DISAGREE,alright how did evolution start ?,See above,1,1,1


In [10]:
data = df.loc[df['sub_both'] == 1]

In [11]:
data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
data['q_end'] = [x+len(y)-1 for x,y in zip(data["q_start"],data["q'"])]
data['r_end'] = [x+len(y)-1 for x,y in zip(data["r_start"],data["r'"])]

<ipython-input-11-2e5ebdb9e59b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['q_start'] = [y.index(x) for x,y in zip(data["q'"],data["q"])]
<ipython-input-11-2e5ebdb9e59b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['r_start'] = [y.index(x) for x,y in zip(data["r'"],data["r"])]
<ipython-input-11-2e5ebdb9e59b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [12]:
data.head()

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both,q_start,r_start,q_end,r_end
0,8,It can go both ways . We all doubt . It is wha...,AGREE:True .,AGREE,It can go both ways . We all doubt . It is wha...,True,1,1,1,0,6,76,9
1,9,"once again , you seem to support the killing o...",AGREE:based on the idea that people are dispen...,AGREE,seem to support the killing of certain people,based on the idea that people are dispensible ...,1,1,1,17,6,61,98
2,10,"I personly would not condone an abortion , how...","AGREE:This is a pretty touchy issue , and I ag...",AGREE,"personly would not condone an abortion , howev...","This is a pretty touchy issue , and I agree wi...",1,1,1,2,6,381,240
4,12,alright how did evolution start ?,DISAGREE:See above,DISAGREE,alright how did evolution start ?,See above,1,1,1,0,9,32,17
5,13,Do you not even get statutory sick pay ?,DISAGREE:You have to be off for more than 4 da...,DISAGREE,Do you not even get statutory sick pay ?,You have to be off for more than 4 days and af...,1,1,1,0,9,39,140


In [13]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(data, test_size=0.2)
valid, test = train_test_split(valid, test_size=0.5)

## Tokenizer

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [15]:
train_data_q = train['q'].tolist()
valid_data_q = valid['q'].tolist()
test_data_q = test['q'].tolist()

train_data_r = train['r'].tolist()
valid_data_r = valid['r'].tolist()
test_data_r = test['r'].tolist()

In [16]:
train_encodings = tokenizer(train_data_q, train_data_r, truncation=True, padding=True)
val_encodings = tokenizer(valid_data_q, valid_data_r, truncation=True, padding=True)
test_encodings = tokenizer(test_data_q, test_data_r, truncation=True, padding=True)

In [17]:
tokenizer.decode(train_encodings['input_ids'][0])

'[CLS] the child in the womb feels pain.. when you put the needle up there to try to give the child anestesia in the womb, the child moves away to not get stuck by the needle. Its because it feels pain.. this is after 18 weeks. this is a child in the womb.. [SEP] DISAGREE : the cricket in the pliers feels pain... When you start to squeeze a cricket with a pair of pliers, it starts to wiggle and tries to get away so as not to get crushed by the pliers. Is it because the cricket feels pain? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [18]:
train_answer = train[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')
valid_answer = valid[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')
test_answer = test[['q_start', 'r_start',	'q_end', 'r_end']].to_dict('records')

In [19]:
def add_token_positions(encodings, answers):
    q_start, r_start, q_end, r_end = [],[],[],[]

    for i in range(len(answers)):
        q_start.append(encodings.char_to_token(i, answers[i]['q_start'], 0))
        r_start.append(encodings.char_to_token(i, answers[i]['r_start'], 1))
        q_end.append(encodings.char_to_token(i, answers[i]['q_end'], 0))
        r_end.append(encodings.char_to_token(i, answers[i]['r_end'], 1))

        if q_start[-1] is None:
            q_start[-1] = 0
            q_end[-1] = 0
            # continue

        if r_start[-1] is None:
            r_start[-1] = 0
            r_end[-1] = 0
            # continue

        shift = 1
        while q_end[-1] is None:
            q_end[-1] = encodings.char_to_token(i, answers[i]['q_end'] - shift)
            shift += 1
        shift = 1
        while r_end[-1] is None:
            r_end[-1] = encodings.char_to_token(i, answers[i]['r_end'] - shift)
            shift += 1
    encodings.update({'q_start':q_start, 'r_start':r_start,	'q_end':q_end, 'r_end':r_end})

In [20]:
# Convert char_based_id to token_based_id
# Find the corossponding token id after input being tokenized
add_token_positions(train_encodings, train_answer)
add_token_positions(val_encodings, valid_answer)
add_token_positions(test_encodings, test_answer)

In [21]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'q_start', 'r_start', 'q_end', 'r_end'])

In [22]:
print(train_encodings['q_start'][0])
print(train_encodings['r_start'][0])
print(train_encodings['q_end'][0])
print(train_encodings['r_end'][0])

1
74
48
128


## Dataset

In [23]:


class qrDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [24]:
train_dataset = qrDataset(train_encodings)
val_dataset = qrDataset(val_encodings)
test_dataset = qrDataset(test_encodings)

In [25]:
next(iter(train_dataset))

{'input_ids': tensor([  101,  1103,  2027,  1107,  1103,   192, 20972,  5115,  2489,   119,
           119,  1165,  1128,  1508,  1103, 13864,  1146,  1175,  1106,  2222,
          1106,  1660,  1103,  2027,  1126, 17563,  6370,  1107,  1103,   192,
         20972,   117,  1103,  2027,  5279,  1283,  1106,  1136,  1243,  5342,
          1118,  1103, 13864,   119,  2098,  1272,  1122,  5115,  2489,   119,
           119,  1142,  1110,  1170,  1407,  2277,   119,  1142,  1110,   170,
          2027,  1107,  1103,   192, 20972,   119,   119,   102,   141,  6258,
         22689, 16941,  2036,   131,  1103,  5428,  1107,  1103,   185, 12996,
          5115,  2489,   119,   119,   119,  1332,  1128,  1838,  1106, 11633,
           170,  5428,  1114,   170,  3111,  1104,   185, 12996,   117,  1122,
          3816,  1106,   192,  6512,  6415,  1105,  4642,  1106,  1243,  1283,
          1177,  1112,  1136,  1106,  1243, 10613,  1118,  1103,   185, 12996,
           119,  2181,  1122,  1272,  1

## Model

In [26]:
from transformers import BertModel

class myModel(torch.nn.Module):

    def __init__(self):

        super(myModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.fc = nn.Linear(768, 4)
        

    def forward(self, input_ids, attention_mask, token_type_ids):

        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, return_dict=True)
        logits = output[0]
        out = self.fc(logits)

        return out



## Training

In [27]:
from transformers import AdamW
from tqdm import tqdm

# Set GPU / CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# Put model on device
model = myModel().to(device)

optim = AdamW(model.parameters(), lr=1e-5)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a f

In [28]:
# Pack data into dataloader by batch
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [29]:
training_epoch = 1

In [30]:
loss_fct = CrossEntropyLoss()

In [31]:
def evaluate(valid_loader):
    model.eval()
    running_loss = 0.0

    with torch.no_grad():
        loop = tqdm(valid_loader, leave=True)
        for batch_id, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            q_start = batch['q_start'].to(device)
            r_start = batch['r_start'].to(device)
            q_end = batch['q_end'].to(device)
            r_end = batch['r_end'].to(device)

            # model output
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

            q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

            q_start_logits = q_start_logits.squeeze(-1).contiguous()
            r_start_logits = r_start_logits.squeeze(-1).contiguous()
            q_end_logits = q_end_logits.squeeze(-1).contiguous()
            r_end_logits = r_end_logits.squeeze(-1).contiguous()

            q_start_loss = loss_fct(q_start_logits, q_start)
            r_start_loss = loss_fct(r_start_logits, r_start)
            q_end_loss = loss_fct(q_end_logits, q_end)
            r_end_loss = loss_fct(r_end_logits, r_end)

            loss = q_start_loss + r_start_loss + q_end_loss + r_end_loss

            running_loss += loss.item()
            if batch_id % 30 == 0 and batch_id != 0:
                print('Validation Epoch {} Batch {} Loss {:.4f}'.format(
                    batch_id + 1, batch_id, running_loss / 30))
                running_loss = 0.0

        print("evaluate loss: ", loss)

In [32]:
for epoch in range(training_epoch):
    model.train()
    running_loss = 0.0

    loop = tqdm(train_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        # reset
        optim.zero_grad()


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        q_start = batch['q_start'].to(device)
        r_start = batch['r_start'].to(device)
        q_end = batch['q_end'].to(device)
        r_end = batch['r_end'].to(device)


        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_loss = loss_fct(q_start_logits, q_start)
        r_start_loss = loss_fct(r_start_logits, r_start)
        q_end_loss = loss_fct(q_end_logits, q_end)
        r_end_loss = loss_fct(r_end_logits, r_end)



        loss = q_start_loss + r_start_loss + q_end_loss + r_end_loss

        # calculate loss
        loss.backward()
        # update parameters
        optim.step()

        running_loss += loss.item()
        if batch_id % 50 == 0 and batch_id != 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                batch_id + 1, batch_id, running_loss / 50))
            running_loss = 0.0

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
    evaluate(valid_loader)

Epoch 0:   8%|▊         | 51/607 [00:39<06:43,  1.38it/s, loss=15.3]

Epoch 51 Batch 50 Loss 19.9860


Epoch 0:  17%|█▋        | 101/607 [01:16<06:21,  1.33it/s, loss=10.8]

Epoch 101 Batch 100 Loss 10.5432


Epoch 0:  25%|██▍       | 151/607 [01:54<05:46,  1.32it/s, loss=6]

...

Validation Epoch 31 Batch 30 Loss 6.1892


 80%|████████  | 61/76 [00:15<00:03,  3.93it/s]

Validation Epoch 61 Batch 60 Loss 5.6575


100%|██████████| 76/76 [00:19<00:00,  3.94it/s]

evaluate loss:  tensor(5.4871, device='cuda:0')


In [34]:
torch.save(model.state_dict(), directory + 'model_datareduce_base')

In [ ]:
# model = myModel().to(device)
# model.load_state_dict(torch.load(directory + 'aicup_model'))

## Predict

In [35]:
def predict(test_loader):
    predict_pos = []

    model.eval()

    q_sub_output, r_sub_output = [],[]

    loop = tqdm(test_loader, leave=True)
    for batch_id, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        # model output
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        
        q_start_logits, r_start_logits, q_end_logits, r_end_logits = torch.split(outputs, 1, 2)

        q_start_logits = q_start_logits.squeeze(-1).contiguous()
        r_start_logits = r_start_logits.squeeze(-1).contiguous()
        q_end_logits = q_end_logits.squeeze(-1).contiguous()
        r_end_logits = r_end_logits.squeeze(-1).contiguous()

        q_start_prdict = torch.argmax(q_start_logits, 1).cpu().numpy()
        r_start_prdict = torch.argmax(r_start_logits, 1).cpu().numpy()
        q_end_prdict = torch.argmax(q_end_logits, 1).cpu().numpy()
        r_end_prdict = torch.argmax(r_end_logits, 1).cpu().numpy()

        for i in range(len(input_ids)):
            predict_pos.append((q_start_prdict[i].item(), r_start_prdict[i].item(), q_end_prdict[i].item(), r_end_prdict[i].item()))

            q_sub = tokenizer.decode(input_ids[i][q_start_prdict[i]:q_end_prdict[i]+1])
            r_sub = tokenizer.decode(input_ids[i][r_start_prdict[i]:r_end_prdict[i]+1])
            
            q_sub_output.append(q_sub)
            r_sub_output.append(r_sub)
    
    return q_sub_output, r_sub_output, predict_pos



In [36]:
q_sub_output, r_sub_output, predict_pos = predict(test_loader)

100%|██████████| 76/76 [00:19<00:00,  3.95it/s]


In [37]:
def get_output_post_fn(test, q_sub_output, r_sub_output):
    q_sub, r_sub = [], []
    for i in range(len(test)):

        q_sub_pred = q_sub_output[i].split()
        r_sub_pred = r_sub_output[i].split()

        if q_sub_pred is None:
            q_sub_pred = []
        q_sub_error_index = q_sub_pred.index('[SEP]') if '[SEP]' in q_sub_pred else -1

        if q_sub_error_index != -1:
            q_sub_pred = q_sub_pred[:q_sub_error_index]

        temp = r_sub_pred.copy()
        if r_sub_pred is None:
            r_sub_pred = []
        else:
            for j in range(len(temp)):
                if temp[j] == '[SEP]':
                    r_sub_pred.remove('[SEP]')
                if temp[j] == '[PAD]':
                    r_sub_pred.remove('[PAD]')

        q_sub.append(' '.join(q_sub_pred))
        r_sub.append(' '.join(r_sub_pred))

    return q_sub, r_sub

In [38]:
q_sub, r_sub = get_output_post_fn(test, q_sub_output, r_sub_output)

In [39]:
test['q_sub'] = q_sub
test['r_sub'] = r_sub

In [40]:
test.head()

,id,q,r,s,q',r',sub_q_true,sub_r_true,sub_both,q_start,r_start,q_end,r_end,q_sub,r_sub
599,768,Gun Lobby Promoted `` Final War `` Against Fed...,"DISAGREE:Oh boy , is this what the Anti-gun cr...",DISAGREE,"In the period leading up to the bombing , viol...","Oh boy , is this what the Anti-gun crowd has c...",1,1,1,585,9,888,62,Gun Lobby Promoted ` ` Final War ` ` Against F...,
7179,9042,The `` evidence for the answer `` is the origi...,DISAGREE:So you 're just ignoring the counters...,DISAGREE,The `` evidence for the answer `` is the origi...,How are they any more irrational than basing a...,1,1,1,0,101,122,271,The ` ` evidence for the answer ` ` is the ori...,So you're just ignoring the counters I provide...
1836,2236,"Actually , one . After the adoptions years ago...",DISAGREE:There may very well be such things ne...,DISAGREE,"Somehow , no matter how there is no money and ...","There may very well be such things near you , ...",1,1,1,214,9,398,167,"Actually, one. After the adoptions years ago, ...","There may very well be such things near you, b..."
1053,1278,The terrorists saw it as absolutly necessary ....,DISAGREE:This seems to conflict with some of y...,DISAGREE,"I disagree . When we kill children in Iraq , w...",This seems to conflict with some of your earli...,1,1,1,60,9,245,95,The terrorists saw it as absolutly necessary. ...,This seems to conflict with some of your earli...
795,999,If you personally choose to live your life in ...,DISAGREE:It 's amazing how little you godless ...,DISAGREE,If you personally choose to live your life in ...,It 's amazing how little you godless heathens ...,1,1,1,0,9,164,209,If you personally choose to live your life in ...,It's amazing how little you godless heathens h...


## Grading

In [41]:
def nltk_token_string(sentence):
    # print(sentence)
    tokens = nltk.word_tokenize(sentence)
    for i in range(len(tokens)):
        if len(tokens[i]) == 1:
            tokens[i] = re.sub(r"[!\"#$%&\'()*\+, -.\/:;<=>?@\[\\\]^_`{|}~]", '', tokens[i])
    while '' in tokens:
        tokens.remove('')
    tokens = ' '.join(tokens)
    return tokens

In [42]:
def lcs(X, Y):
    X_, Y_ = [], []
    
    X_ = nltk_token_string(X)
    Y_ = nltk_token_string(Y)

    m = len(X_)
    n = len(Y_)
 
    # declaring the array for storing the dp values
    L = [[None]*(n + 1) for i in range(m + 1)]
 
    """Following steps build L[m + 1][n + 1] in bottom up fashion
    Note: L[i][j] contains length of LCS of X[0..i-1]
    and Y[0..j-1]"""
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0 :
                L[i][j] = 0
            elif X_[i-1] == Y_[j-1]:
                L[i][j] = L[i-1][j-1]+1
            else:
                L[i][j] = max(L[i-1][j], L[i][j-1])
 
    # L[m][n] contains the length of LCS of X[0..n-1] & Y[0..m-1]
    return L[m][n]


def acc(full, sub):
    common = lcs(full, sub)
    union = len(full) + len(sub) - common
    accuracy = float(common/union)

    return accuracy

In [43]:
q_acc_sum = 0
r_acc_sum = 0
for i in range(test.shape[0]):
    q_accuracy = acc(test.iloc[i]["q'"], test.iloc[i]['q_sub'])
    r_accuracy = acc(test.iloc[i]["r'"], test.iloc[i]['r_sub'])

    q_acc_sum += q_accuracy
    r_acc_sum += r_accuracy

print("q accuracy: ", q_acc_sum/test.shape[0])
print("r accuracy: ", r_acc_sum/test.shape[0])

q accuracy:  0.6804505991232244
r accuracy:  0.6613297764871064
